##### Copyright 2025 Google LLC.

**欢迎来到 Kaggle 5天智能体课程第2天！**

在第1天，你学习了如何创建带有内置工具（如 Google 搜索）的智能体。你还学习了如何编排多智能体系统。现在让我们通过构建自定义逻辑、委托给专业智能体以及处理现实世界的复杂性，来释放智能体工具的全部力量。

为什么智能体需要工具？

没有工具，智能体的知识就被冻结在时间中——它无法获取今天的新闻或你公司的库存信息。它与外部世界没有任何连接，因此智能体无法为你采取行动。

**解决方案：** 工具是将你孤立的 LLM 转变为能够真正帮助你完成任务的强大智能体的关键。

在本笔记本中，你将：

将你的 Python 函数转换为智能体工具
构建一个智能体并将其**作为工具**在另一个智能体中使用
**构建你的第一个多工具智能体**
探索 ADK 中不同的工具类型

In [1]:
from google.genai import types

from google.adk.agents import Agent
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


辅助函数

辅助函数，用于打印生成的 Python 代码以及代码执行工具的结果：

In [2]:
def show_python_code_and_result(response):
    for i in range(len(response)):
        # 检查响应是否包含来自代码执行器的有效函数调用结果
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])


print("✅ Helper functions defined.")

✅ Helper functions defined.


什么是自定义工具？

**自定义工具**是使用你自己的代码和业务逻辑构建的工具。与 ADK 提供的现成内置工具不同，自定义工具让你对功能拥有完全的控制权。

**何时使用自定义工具？**

像 Google 搜索这样的内置工具很强大，但**每个企业都有独特的需求**，通用工具无法处理。自定义工具让你能够实现特定的业务逻辑、连接到你的系统，并解决特定领域的问题。ADK 提供了多种自定义工具类型来处理这些场景。

构建自定义函数工具

示例：货币转换智能体

这个智能体可以将一种货币转换为另一种货币，并计算转换的费用。该智能体有两个自定义工具，遵循以下工作流程：

1. 费用查询工具 - 查找转换的交易费用（模拟）
2. 汇率工具 - 获取货币转换汇率（模拟）
3. 计算步骤 - 计算包括费用在内的总转换成本

<img src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day2/currency-agent.png" width="600" alt="Currency Converter Agent">

如何定义工具？

任何 Python 函数都可以成为智能体工具，只需遵循以下简单指南：

1. 创建一个 Python 函数
2. 遵循下面列出的最佳实践
3. 将你的函数添加到智能体的 `tools=[]` 列表中，ADK 会自动处理其余部分。

ADK 最佳实践的实际应用

注意我们的工具如何遵循 ADK 最佳实践：

**1. 字典返回**：工具返回 `{"status": "success", "data": ...}` 或 `{"status": "error", "error_message": ...}`  
**2. 清晰的文档字符串**：LLM 使用文档字符串来理解何时以及如何使用工具  
**3. 类型提示**：使 ADK 能够生成正确的模式（`str`、`dict` 等）  
**4. 错误处理**：结构化的错误响应帮助 LLM 优雅地处理失败  

In [3]:
# 注意文档字符串、类型提示和返回值。
def get_fee_for_payment_method(method: str) -> dict:
    """Looks up the transaction fee percentage for a given payment method.

    This tool simulates looking up a company's internal fee structure based on
    the name of the payment method provided by the user.

    Args:
        method: The name of the payment method. It should be descriptive,
                e.g., "platinum credit card" or "bank transfer".

    Returns:
        Dictionary with status and fee information.
        Success: {"status": "success", "fee_percentage": 0.02}
        Error: {"status": "error", "error_message": "Payment method not found"}
    """
    # This simulates looking up a company's internal fee structure.
    fee_database = {
        "platinum credit card": 0.02,  # 2%
        "gold debit card": 0.035,  # 3.5%
        "bank transfer": 0.01,  # 1%
    }

    fee = fee_database.get(method.lower())
    if fee is not None:
        return {"status": "success", "fee_percentage": fee}
    else:
        return {
            "status": "error",
            "error_message": f"Payment method '{method}' not found",
        }


print("✅ Fee lookup function created")
print(f"💳 Test: {get_fee_for_payment_method('platinum credit card')}")

✅ Fee lookup function created
💳 Test: {'status': 'success', 'fee_percentage': 0.02}


让我们遵循相同的最佳实践来定义我们的第二个工具 `get_exchange_rate`。

In [4]:
def get_exchange_rate(base_currency: str, target_currency: str) -> dict:
    """Looks up and returns the exchange rate between two currencies.

    Args:
        base_currency: The ISO 4217 currency code of the currency you
                       are converting from (e.g., "USD").
        target_currency: The ISO 4217 currency code of the currency you
                         are converting to (e.g., "EUR").

    Returns:
        Dictionary with status and rate information.
        Success: {"status": "success", "rate": 0.93}
        Error: {"status": "error", "error_message": "Unsupported currency pair"}
    """

    # Static data simulating a live exchange rate API
    # In production, this would call something like: requests.get("api.exchangerates.com")
    rate_database = {
        "usd": {
            "eur": 0.93,  # Euro
            "jpy": 157.50,  # Japanese Yen
            "inr": 83.58,  # Indian Rupee
        }
    }

    # Input validation and processing
    base = base_currency.lower()
    target = target_currency.lower()

    # Return structured result with status
    rate = rate_database.get(base, {}).get(target)
    if rate is not None:
        return {"status": "success", "rate": rate}
    else:
        return {
            "status": "error",
            "error_message": f"Unsupported currency pair: {base_currency}/{target_currency}",
        }


print("✅ Exchange rate function created")
print(f"💱 Test: {get_exchange_rate('USD', 'EUR')}")

✅ Exchange rate function created
💱 Test: {'status': 'success', 'rate': 0.93}


创建货币智能体

关键点：
- `tools=[]` 列表告诉智能体它可以使用哪些函数
- 指令通过精确的函数名称引用工具（例如，
`get_fee_for_payment_method()`）
- 智能体使用这些名称来决定何时以及如何调用每个工具

In [7]:
from google.adk.models.lite_llm import LiteLlm

# Currency agent with custom function tools
currency_agent = LlmAgent(
    name="currency_agent",
    model=LiteLlm(
        model="deepseek/deepseek-chat"
        ),
    instruction="""You are a smart currency conversion assistant.

    For currency conversion requests:
    1. Use `get_fee_for_payment_method()` to find transaction fees
    2. Use `get_exchange_rate()` to get currency conversion rates
    3. Check the "status" field in each tool's response for errors
    4. Calculate the final amount after fees based on the output from `get_fee_for_payment_method` and `get_exchange_rate` methods and provide a clear breakdown.
    5. First, state the final converted amount.
        Then, explain how you got that result by showing the intermediate amounts. Your explanation must include: the fee percentage and its
        value in the original currency, the amount remaining after the fee, and the exchange rate used for the final conversion.

    If any tool returns status "error", explain the issue to the user clearly.
    """,
    tools=[get_fee_for_payment_method, get_exchange_rate],
)

print("✅ Currency agent created with custom function tools")
print("🔧 Available tools:")
print("  • get_fee_for_payment_method - Looks up company fee structure")
print("  • get_exchange_rate - Gets current exchange rates")

✅ Currency agent created with custom function tools
🔧 Available tools:
  • get_fee_for_payment_method - Looks up company fee structure
  • get_exchange_rate - Gets current exchange rates


In [8]:
# Test the currency agent
currency_runner = InMemoryRunner(agent=currency_agent)
_ = await currency_runner.run_debug(
    "I want to convert 500 US Dollars to Euros using my Platinum Credit Card. How much will I receive?"
)


 ### Created new session: debug_session_id

User > I want to convert 500 US Dollars to Euros using my Platinum Credit Card. How much will I receive?
currency_agent > I'll help you convert 500 US Dollars to Euros using your Platinum Credit Card. First, I need to get the transaction fee for your payment method and then the exchange rate.
currency_agent > Now let me get the exchange rate from USD to EUR:
currency_agent > **Final converted amount: €455.70**

Here's how I calculated this:

1. **Original amount:** $500.00 USD
2. **Transaction fee:** 2% (from Platinum Credit Card)
   - Fee amount: $500.00 × 0.02 = $10.00 USD
3. **Amount after fee:** $500.00 - $10.00 = $490.00 USD
4. **Exchange rate:** 1 USD = 0.93 EUR
5. **Final conversion:** $490.00 × 0.93 = €455.70

So after the 2% transaction fee ($10.00) and conversion at the rate of 0.93 EUR per USD, you will receive €455.70.


使用代码提高智能体可靠性

智能体的指令说*"计算扣除费用后的最终金额"*，但 LLM 在数学计算方面并不总是可靠的。它们可能会犯计算错误或使用不一致的公式。

解决方案：让我们要求智能体生成 Python 代码来进行数学计算，然后运行它以给我们最终结果！代码执行比让 LLM 在脑海中尝试做数学计算要可靠得多！

<img src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day2/enhanced-currency-agent.png" width="800" alt="Enhanced Currency Converter Agent">

内置代码执行器

ADK 有一个内置的代码执行器，能够在沙箱中运行代码。

让我们创建一个 `calculation_agent`，它接收 Python 代码并使用 `BuiltInCodeExecutor` 来运行它。

In [9]:
calculation_agent = LlmAgent(
    name="CalculationAgent",
    model=LiteLlm(model="deepseek/deepseek-chat"),
    instruction="""You are a specialized calculator that ONLY responds with Python code. You are forbidden from providing any text, explanations, or conversational responses.
 
     Your task is to take a request for a calculation and translate it into a single block of Python code that calculates the answer.
     
     **RULES:**
    1.  Your output MUST be ONLY a Python code block.
    2.  Do NOT write any text before or after the code block.
    3.  The Python code MUST calculate the result.
    4.  The Python code MUST print the final result to stdout.
    5.  You are PROHIBITED from performing the calculation yourself. Your only job is to generate the code that will perform the calculation.
   
    Failure to follow these rules will result in an error.
       """,
    code_executor=BuiltInCodeExecutor(),  # Use the built-in Code Executor Tool. This gives the agent code execution capabilities
)

更新智能体的指令和工具集

1. 更新 `currency_agent` 的指令以生成 Python 代码
- 原始："*计算扣除费用后的最终金额*"（模糊的数学指令）
- 增强："*生成 Python 代码来计算最终金额...并使用 `calculation_agent` 运行代码并计算最终金额*"

2. 将 `calculation_agent` 添加到工具集

    ADK 允许你使用 `AgentTool` 将任何智能体作为工具使用。

- 将 `AgentTool(agent=calculation_agent)` 添加到工具列表
- 专业智能体作为可调用工具出现在根智能体中


In [10]:
enhanced_currency_agent = LlmAgent(
    name="enhanced_currency_agent",
    model=LiteLlm(model="deepseek/deepseek-chat"),
    # Updated instruction
    instruction="""You are a smart currency conversion assistant. You must strictly follow these steps and use the available tools.

  For any currency conversion request:

   1. Get Transaction Fee: Use the get_fee_for_payment_method() tool to determine the transaction fee.
   2. Get Exchange Rate: Use the get_exchange_rate() tool to get the currency conversion rate.
   3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
   4. Calculate Final Amount (CRITICAL): You are strictly prohibited from performing any arithmetic calculations yourself. You must use the calculation_agent tool to generate Python code that calculates the final converted amount. This 
      code will use the fee information from step 1 and the exchange rate from step 2.
   5. Provide Detailed Breakdown: In your summary, you must:
       * State the final converted amount.
       * Explain how the result was calculated, including:
           * The fee percentage and the fee amount in the original currency.
           * The amount remaining after deducting the fee.
           * The exchange rate applied.
    """,
    tools=[
        get_fee_for_payment_method,
        get_exchange_rate,
        AgentTool(agent=calculation_agent),  # Using another agent as a tool!
    ],
)

print("✅ Enhanced currency agent created")
print("🎯 New capability: Delegates calculations to specialist agent")
print("🔧 Tool types used:")
print("  • Function Tools (fees, rates)")
print("  • Agent Tool (calculation specialist)")

✅ Enhanced currency agent created
🎯 New capability: Delegates calculations to specialist agent
🔧 Tool types used:
  • Function Tools (fees, rates)
  • Agent Tool (calculation specialist)


In [11]:
# Define a runner
enhanced_runner = InMemoryRunner(agent=enhanced_currency_agent)

In [12]:
# Test the enhanced agent
response = await enhanced_runner.run_debug(
    "Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation."
)


 ### Created new session: debug_session_id

User > Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation.
enhanced_currency_agent > I'll help you convert 1,250 USD to INR using a Bank Transfer. Let me follow the required steps:

## Step 1: Get Transaction Fee for Bank Transfer
enhanced_currency_agent > Good! The transaction fee for Bank Transfer is 1% (0.01).

## Step 2: Get Exchange Rate from USD to INR
enhanced_currency_agent > Excellent! The exchange rate is 1 USD = 83.58 INR.

## Step 3: Calculate Final Amount

Now I'll use the calculation agent to compute the final converted amount:


ValueError: Gemini code execution tool is not supported for model deepseek/deepseek-chat

当货币智能体调用 `CalculationAgent` 时，它传递了生成的 Python 代码
`CalculationAgent` 接着使用 `BuiltInCodeExecutor` 运行代码，并给了我们精确的计算结果，而不是 LLM 的猜测！

In [14]:
show_python_code_and_result(response)

Generated Python Code >>  
original_amount_usd = 1250
fee_percentage = 1.0
exchange_rate = 83.58

# Calculate the fee amount
fee_amount_usd = original_amount_usd * (fee_percentage / 100)

# Calculate the amount after deducting the fee
amount_after_fee_usd = original_amount_usd - fee_amount_usd

# Calculate the final converted amount in INR
final_amount_inr = amount_after_fee_usd * exchange_rate

print(f"Original Amount (USD): {original_amount_usd}")
print(f"Fee Percentage: {fee_percentage}%")
print(f"Fee Amount (USD): {fee_amount_usd:.2f}")
print(f"Amount after deducting fee (USD): {amount_after_fee_usd:.2f}")
print(f"Exchange Rate (INR/USD): {exchange_rate}")
print(f"Final Converted Amount (INR): {final_amount_inr:.2f}")



智能体工具 vs 子智能体：有什么区别？

这是一个常见的问题！两者都涉及使用多个智能体，但它们的工作方式非常不同：

**智能体工具：**
- 智能体 A 将智能体 B 作为工具调用
- 智能体 B 的响应返回给智能体 A
- 智能体 A 保持控制权并继续对话
使用场景：委托特定任务（如计算）

**子智能体（不同的模式）：**
- 智能体 A 将控制权完全转移给智能体 B
- 智能体 B 接管并处理所有未来的用户输入
- 智能体 A 退出循环
使用场景：移交给专家（如客户支持层级）

**在我们的货币示例中：** 我们希望货币智能体获取计算结果并继续使用它们，所以我们使用**智能体工具**，而不是子智能体。

**ADK 工具类型完整指南**

大致分为两类：**自定义工具**和**内置工具**

1. 自定义工具

<img src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day2/custom-tools.png" width="800" alt="Custom Tools">

是什么：你为特定需求自己构建的工具

优势：对功能拥有完全控制权——你可以构建智能体确切需要的东西

函数工具<br> 
是什么：转换为智能体工具的 Python 函数<br>
示例：`get_fee_for_payment_method`、`get_exchange_rate`<br>
优势：立即将任何 Python 函数转换为智能体工具<br>

长时间运行函数工具<br>
是什么：用于需要大量时间的操作的函数<br>
示例：人工审批、文件处理<br>
优势：智能体可以启动任务并在等待时继续其他工作

智能体工具<br>
是什么：用作工具的其他智能体<br>
示例：`AgentTool(agent=calculation_agent)`<br>
优势：构建专业智能体并在不同系统中重用它们

MCP 工具<br>
是什么：来自模型上下文协议服务器的工具<br>
示例：文件系统访问、Google Maps、数据库<br>
优势：连接到任何 MCP 兼容的服务，无需自定义集成

OpenAPI 工具<br>
是什么：从 API 规范自动生成的工具<br>
示例：REST API 端点变为可调用工具<br>
优势：无需手动编码——只需提供 API 规范即可获得可用的工具

2. 内置工具

<img src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day2/built-in-tools.png" width="1200" alt="Built-in Tools">

是什么：ADK 提供的预构建工具

优势：无需开发时间——零设置即可立即使用

Gemini 工具<br>
是什么：利用 Gemini 功能的工具<br>
示例：`google_search`、`BuiltInCodeExecutor`<br>
优势：可靠、经过测试的工具，开箱即用

Google Cloud 工具<br>
是什么：用于 Google Cloud 服务和企业集成的工具<br>
示例：`BigQueryToolset`、`SpannerToolset`、`APIHubToolset`<br>
优势：企业级数据库和 API 访问，内置安全性

第三方工具<br>
是什么：现有工具生态系统的包装器<br>
示例：Hugging Face、Firecrawl、GitHub 工具<br>
优势：重用现有工具投资——无需重建已存在的东西